#  Install package

In [117]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install dash

# Import package

In [118]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
import openai


# Define Solution class
Please run the following cell to define the functions

In [119]:
# %load_ext autoreload
# %autoreload 2

import LLM_Geo_Constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from LLM_Geo_kernel import Solution

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Demonstration Cases

## Input task and data desciption

In [120]:
# Case 1: population living near hazardous wastes

'''
TASK = r"""1) Find out the total population that lives within a tract that contain hazardous waste facilities. The study area is North Carolina, US.
2) Generate a map to show the spatial distribution of population at the tract level and highlight the borders of tracts that have hazardous waste facilities.
"""

DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file location: https://github.com/gladcolor/LLM- Geo/raw/master/overlay_analysis/Hazardous_Waste_Sites.zip.",
                  "NC tract boundary shapefile location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_shp_37.zip. The tract id column is 'Tract'.",
                  "NC tract population CSV file location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID'."
                 ]


task_name ='Resident_at_risk_counting'
'''

# Case 2: mobility data retrieval and visulization
"""
TASK = r'''
1) Show the monthly change rates of each administrative regions in a France map. Each month is a sub-map in a map matrix. The base of the change rate is January 2020. 
2) Draw a line chart to show the monthly change rate trends of all administrative regeions.

'''

DATA_LOCATIONS = ["ESRI shapefile for France administrative regions:" + \
                  "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip." + \
                  "The 'GID_1' column is the administrative region code, 'NAME_1' column is the administrative region name.",
                  "REST API url with parameters for mobility data access:" + \
                  "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020." + \
                  "The response is in CSV format. There are three columns in the response: " + \
                  "place,date (format:2020-01-07), and intra_movement. 'place' column is the administractive region code, France administrative regions start with 'FRA'.",
                 ]


# task_name ='France_mobility_changes_2020'
"""

# Case 3: Provider address extraction
# TASK = r"""1) Find out the Autism service providers' addresses or location from their website. The address usually listed in the homepage, 'about' or 'contact' page. The latter two pages usually contain 'about' or 'contact' in links embedded in the homepage.
# 2) A provider may have multiple service address. If cannot find the address, simply return an empty text, DO NOT make up fake addresses. 
# 3) You need to send webpage text ChatGPT to extract address. Use this pre-written function your designed detailed prompt to get response from ChatGPT: helper.get_LLM_reply(prompt=your_prompt_with_webpage_text, model=r"gpt-4",). Use this statement to extract content from the returned response: response['choices'][0]['message']['content']. Let ChatGPT reply in json format as {'address': extracted_address}.  DO NOT reuturn explaination or conversation, return the address or empty text only. 
# 4) Save the extracted addresses as "Address" column, together with the given 'Provider' and 'Web Site' columns. If there are multile addresses for a provider, each address is a row in the CSV file.
# """

# DATA_LOCATIONS = ["Autism service provider webpage file location: E:\Research\LLM-Geo\Address_extraction\ACE_providers_AGIS.csv. The 'Web Site' column is the URL, the 'Provider' column is the provider name.",                  
#                  ]

# task_name ='Address_extraction'




"""
# Case 4: COVID-19 prevalence mapping (Testing, not ready yet)
TASK = r'''1) Draw a map matrix of South Carolina counties' monthly COVID-19 infection ratio in 2021. Each month is a submap.
2) county infection ratio = (infection of this month / county popultion).
'''

DATA_LOCATIONS = [r"South Carolina county boudary, ESRI shapefile: https://github.com/gladcolor/LLM-Geo/raw/master/COVID-19/SC_counties.zip. ",
                  r"COVID-19 data case in 2021 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2021.csv. It is a CSV file; there are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",                  
                  r"Population data: use Python library CensusData to obtain data. ",
                 ]
# API_DOC_LOCATION = [(1, r'https://raw.githubusercontent.com/gladcolor/LLM-Geo/master/COVID-19/CensusData_API_DOC.txt')] 
API_DOC_LOCATION = [(2, r'./COVID-19/CensusData_API_DOC.txt')] 
# [(Input_data_index, API_cocumentation_path)]


# add the API documentation to DATA_LOCATION
for idx, path in API_DOC_LOCATION:
    with open(path, 'r', encoding='utf-8') as f:
        docs = f.readlines()
    docs = '\n'.join(docs)

    DATA_LOCATIONS[idx] += "The documentation is: \n" + docs


# https://raw.githubusercontent.com/gladcolor/LLM-Geo/master/Address_extraction/ACE_providers_AGIS.csv

# TASK = r'''1)Retrieve the data from the REST API and plot the intra_movement column of the returned data as line chart to show the temporal trend of all states. 
# 2) plot the temporal trend of the movement for each state. Each state figure will be sub figure in the plot. The plot has 5 columns. In addition, please add a weekly smoothed line to each sub plot, and change the line color to orange.
# 3) Using the REST API with date range from 01/01/2020 to 12/31/2020 to analyze the movement reduction rate for each state during two periods: the first period is 01/01/2020-02/29/2020, second period is 03/01/2020 to 04/30/2020. Please find out the reduction rate for each state during the two periods, and create a table to report the result with two columns: state name, reduction rate, sorted by reduction rate.
# '''
# '''
# DATA_LOCATIONS = ["REST API url with parameters for data access: http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=us_state&begin=01/01/2020&end=12/31/2020; The response is in CSV format. There are three columns in the response: place,date,intra_movement; place refers to the state name."
#                  ]
# '''
# 3) Show the administrative region name in the map and chart.
#
# task_name ='COVID-19_infection_rate'
'''

"""

"""
# Case 5: interactive visualization
#Example Source: https://artificialcorner.com/insanely-fast-data-storytelling-with-chatgpt-and-python-1bddae3976f3

task_name ='interactive_visualization'

TASK = r''' Use Plotly to create interactive map and charts.
1) Create a Plotly interactive map to show each country's average per capita CO2 emission between 1970 and 2020. 
2) Select the top 10 countries by total emissions between 1970 and 2020, then draw a stacked area chart using Plotly to show their annual trends between 1970 and 2020.
3) Select the top 10 countries by average per capita emission between 1970 and 2020, then draw a line chart using Plotly to show their annual trends between 1970 and 2020.
'''

DATA_LOCATIONS = [
                  r"CO2 emission CSV file: https://github.com/GIBDUSC/test/raw/master/CO2_emission.csv. The needed columns are: 'Country' , 'Year', 'Total', and 'Per Capita'.",  
                  r"Country boundary ESRI Shapefile: https://github.com/gladcolor/LLM-Geo/raw/develop/interactive_visualization/world_countries.zip. The country name is in the 'name' attribute."
]

# The map project is EPSG:3857 althought not set.
"""


# Case 3: COVID-19 prevalence trend
task_name ='COVID_death_rate'
TASK = r'''1) Draw a map to show the death rate (death/case) of COVID-19 among the countiguous US counties in 2020. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate.
2) The map size is 15*10 inches.  
3) Draw a scatter chart to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value.
'''

DATA_LOCATIONS = [
                  r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. There are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",   
                  r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip. The county FIPS column is 'GEOID'. ",
                  r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county_cleaned.csv. THe needed columns are: 'FIPS', 'Total Population', 'Total Population: 65 to 74 Years', 'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'.",
                  # r"Census data can be obtained use censusdata library. The returned pandas dataframe has a index like: Fayette County, Illinois: Summary level: 050, state:17> county:051. You need to write a function to extract county FIPS. ",
                 ]



save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
# model=r"gpt-3.5-turbo"
model=r"gpt-4"
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.graph_prompt)

Prompt to get solution graph:

Your role: A professional Geo-information scientist and developer good at Python. 
Task: Generate a graph (data structure) only, whose nodes are (1) a series of consecutive steps and (2) data to solve this question:  
 1) Draw a map to show the death rate (death/case) of COVID-19 among the countiguous US counties in 2020. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate.
2) The map size is 15*10 inches.  
3) Draw a scatter chart to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value.
 
Your reply needs to meet these requirements: 
 1. Think step by step.
2. Steps and data (both input and output) form a graph stored in NetworkX. Disconnected components are NOT allowed.
3. Each step is a data process operation: the input can be data paths or variables, and the output can be data paths or variables.
4. There are two types of nodes: a) operation node, and b) data node

## Get graph code from GPT API

In [121]:
response_for_graph = solution.get_LLM_response_for_graph() 
solution.graph_response = response_for_graph
solution.save_solution()
print()
print("Code to generate solution graph: \n")
print(solution.code_for_graph)

Geting LLM reply...
Got LLM reply.

Code to generate solution graph: 

import networkx as nx

G = nx.DiGraph()

#1 Load COVID-19 data
G.add_node("covid_data_url", node_type="data", data_path="https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv", description="Daily accumulated COVID cases and deaths in the US counties for 2020")
G.add_node("load_covid_data", node_type="operation", description="Load COVID-19 data from CSV")
G.add_edge("covid_data_url", "load_covid_data")
G.add_node("covid_dataframe", node_type="data", description="COVID-19 cases and deaths DataFrame")
G.add_edge("load_covid_data", "covid_dataframe")

#2 Load county boundary data
G.add_node("county_boundary_url", node_type="data", data_path="https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip", description="Contiguous US county boundaries shapefile")
G.add_node("load_county_boundary", node_type="operation", description="Load county boundary data from ESRI shapefile")
G.add_edg

## Execute code to generate the solution graphto generate the solution graph

In [122]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

E:\Research\LLM-Geo\COVID_death_rate.html


## Generate prompts and code for operations (functions)

In [ ]:
operations = solution.get_LLM_responses_for_operations()
solution.save_solution()

# all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])
# print("All operation code: \n")
# print(all_operation_code_str)

1 / 9, load_covid_data
Geting LLM reply...


In [ ]:
all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])
print(all_operation_code_str)
exec(all_operation_code_str)


In [ ]:
# ! pip install altair
# ! pip install panel

## Generate prompts and code for assembly program

In [ ]:
assembly_LLM_response = solution.get_LLM_assembly_response()
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

# print("Assembly code: \n")
# print(solution.code_for_assembly)

## Execute assembly code

In [ ]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly
print(solution.code_for_assembly)

In [ ]:
print(all_code)

In [ ]:
exec(all_code)

In [ ]:
stop

In [ ]:
import pandas as pd

def load_covid_data(us_counties_covid_data='https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv'):
    """
    Description: Load COVID-19 cases and deaths data
    
    Args:
        us_counties_covid_data (str): URL of the COVID-19 cases and deaths data CSV file
        
    Returns:
        covid_df (DataFrame): DataFrame containing the loaded COVID-19 data
    """
    covid_df = pd.read_csv(us_counties_covid_data, dtype={'fips': 'str'})
    return covid_df
import geopandas as gpd

def load_county_boundary(us_county_boundary='https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip'):
    """
    Description: Load contiguous US county boundary shapefile

    Args:
    us_county_boundary (str): URL of the county boundary shapefile

    Returns:
    county_gdf (GeoDataFrame): Geopandas GeoDataFrame containing US county boundaries
    """
    county_gdf = gpd.read_file(us_county_boundary)
    return county_gdf
import pandas as pd

def load_census_data(us_census_data='https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county_cleaned.csv'):
    """
    Load US census data.
    
    Arguments:
    - us_census_data: URL for the census CSV data file. Default is https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county_cleaned.csv.

    Returns:
    - census_df: Pandas DataFrame containing the required columns from the census CSV data, with FIPS column as string type.
    """

    # Read the CSV file while converting the FIPS column to string type without leading zeros.
    census_df = pd.read_csv(us_census_data,
                             dtype={'FIPS': 'str'},
                             usecols=['FIPS', 'Total Population', 'Total Population: 65 to 74 Years',
                                      'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'])

    # Drop any rows with missing data and report the number of dropped rows
    missing_count = census_df.isna().sum().sum()
    if missing_count > 0:
        print(f"Dropped {missing_count} rows with missing data.")
        census_df = census_df.dropna()

    return census_df
def filter_last_day_covid(covid_df):
    """
    Description: Filter COVID-19 data by the last day of 2020 (2020-12-31)
    
    Args:
        covid_df (DataFrame): DataFrame containing the loaded COVID-19 data
      
    Returns:
        last_day_covid (DataFrame): DataFrame containing the COVID-19 data for 2020-12-31
    """
    last_day_covid = covid_df[covid_df['date'] == '2020-12-31']
    return last_day_covid
def calculate_death_rate(last_day_covid):
    """
    Description: Calculate the death rate for each county

    Args:
        last_day_covid (DataFrame): DataFrame containing the COVID-19 data for 2020-12-31
                  
    Returns:
        death_rates (DataFrame): DataFrame containing the calculated death rates for each county
    """
    last_day_covid['death_rate'] = last_day_covid['deaths'] / last_day_covid['cases']
    death_rates = last_day_covid[['fips', 'death_rate']]
    return death_rates
import pandas as pd
import geopandas as gpd

def join_covid_boundary(county_gdf, death_rates):
    """
    Description: Join the COVID-19 data and county boundaries using FIPS

    Args:
        county_gdf (GeoDataFrame): Geopandas GeoDataFrame containing US county boundaries
        death_rates (DataFrame): DataFrame containing the calculated death rates for each county

    Returns:
        covid_boundary_gdf (GeoDataFrame): Geopandas GeoDataFrame containing US county boundaries with death rates
    """
    # Convert the 'GEOID' column from new county boundary GeoDataFrame to string without leading zeros
    county_gdf['GEOID'] = county_gdf['GEOID'].astype('str')

    # Merge county boundaries GeoDataFrame with death_rates DataFrame on 'GEOID' and 'fips' columns
    covid_boundary_gdf = pd.merge(county_gdf, death_rates, left_on='GEOID', right_on='fips')

    # Keep only necessary columns in the merged GeoDataFrame
    covid_boundary_gdf = covid_boundary_gdf[['GEOID', 'geometry', 'death_rate']]

    return covid_boundary_gdf
def calculate_senior_rate(county_gdf, census_df):
    """
    Calculate the senior resident rate for each county.

    Args:
    county_gdf (GeoDataFrame): Geopandas GeoDataFrame containing US county boundaries
    census_df (DataFrame): Pandas DataFrame containing the required columns from the census CSV data

    Returns:
    senior_rates (DataFrame): Pandas DataFrame containing county FIPS and calculated senior resident rate
    """

    # Merge the county boundary and census data based on FIPS and GEOID
    merged_data = county_gdf.merge(census_df, left_on='GEOID', right_on='FIPS', how='inner')

    # Calculate the senior resident rate
    merged_data['Senior_Rate'] = ((merged_data['Total Population: 65 to 74 Years'] +
                                  merged_data['Total Population: 75 to 84 Years'] +
                                  merged_data['Total Population: 85 Years and Over']) /
                                  merged_data['Total Population']) * 100

    # Create a DataFrame containing county FIPS and the senior resident rate
    senior_rates = merged_data[['FIPS', 'Senior_Rate']].copy()

    return senior_rates
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import linregress

def draw_death_rate_map(covid_boundary_gdf):
    """
    Description: Draw a map to show the COVID-19 death rate among US counties

    Args:
        covid_boundary_gdf (GeoDataFrame): Geopandas GeoDataFrame containing US county boundaries with death rates

    Returns:
        death_rate_map (matplotlib.figure.Figure): Map showing the death rate in each US county
    """

    # Set the figure size
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))

    # Set color bar size
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    # Plot the map with COVID-19 death rate
    death_rate_map = covid_boundary_gdf.plot(column='death_rate', ax=ax, legend=True, cax=cax,
                                             cmap='viridis', vmin=0, vmax=0.15,
                                             legend_kwds={'label': 'Death Rate'})

    # Set map aesthetics
    ax.set_title('COVID-19 Death Rate among US Counties (2020)')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_axis_off()

    return death_rate_map
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
import numpy as np

def create_scatter_chart(death_rates, senior_rates):
    """
    Description: Create a scatter chart to show the correlation and trend line of death rate and senior resident rate
    
    Args:
        death_rates (DataFrame): DataFrame containing the death rates for each county
        senior_rates (DataFrame): DataFrame containing the senior resident rates for each county
        
    Returns:
        scatter_chart (matplotlib.figure.Figure): Scatter chart with the correlation and trend line of death rate and senior resident rate
    """
    merged_data = death_rates.merge(senior_rates, left_on='fips', right_on='FIPS', how='inner')

    slope, intercept, r_value, p_value, std_err = linregress(merged_data['Senior_Rate'], merged_data['death_rate'])

    line = slope * merged_data['Senior_Rate'] + intercept

    # Create the scatter chart
    plt.figure(figsize=(15, 10))
    scatter_chart = sns.scatterplot(data=merged_data, x='Senior_Rate', y='death_rate', alpha=0.5)
    plt.plot(merged_data['Senior_Rate'], line, color='red', linestyle='-')

    plt.xlabel('Senior Resident Rate (%)')
    plt.ylabel('Death Rate Ratio')
    plt.title(f"Correlation and Trend Line of Death Rate and Senior Resident Rate\nR-squared = {r_value**2:.3f}, p-value = {p_value:.3f}")

    return scatter_chart.figure
 
    # Load the required datasets
covid_df = load_covid_data()
county_gdf = load_county_boundary()
census_df = load_census_data()

# Filter COVID-19 data for the last day of 2020 (2020-12-31)
last_day_covid = filter_last_day_covid(covid_df)

# Calculate the death rate for each county
death_rates = calculate_death_rate(last_day_covid)

# Join the COVID-19 data and county boundaries using FIPS
covid_boundary_gdf = join_covid_boundary(county_gdf, death_rates)

# Calculate the senior resident rate for each county
senior_rates = calculate_senior_rate(county_gdf, census_df)

# Step 1: Draw a map to show the COVID-19 death rate among US counties
death_rate_map = draw_death_rate_map(covid_boundary_gdf)
# death_rate_map.savefig('death_rate_map.png', dpi=300)

# Step 3: Create a scatter chart to show the correlation and trend line of death rate and senior resident rate
scatter_chart = create_scatter_chart(death_rates, senior_rates)
# scatter_chart.savefig('scatter_chart.png', dpi=300)

In [ ]:
census_data.isna().sum()#.sum()

In [ ]:
# census_data_url='https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county_cleaned.csv'
census_data_url = r"E:\OneDrive_USC\OneDrive - University of South Carolina\Research\Spatial_data\ACS2021_5year_county_cleaned.csv"
census_dataframe = pd.read_csv(census_data_url, dtype={'GEOID': str})  # , encoding='ISO-8859-1'
census_dataframe
rows_with_nan = census_dataframe.isna().any(axis=1)
rows_with_nan
print(rows_with_nan.sum())
census_dataframe[rows_with_nan]

In [ ]:
df =  census_dataframe[rows_with_nan]

df

In [ ]:
nans = df.isna()

# Print the row and column indices
for column in nans.columns:
    for idx, is_nan in enumerate(nans[column]):
        if is_nan:
            print(f"NaN found in column {column}, row {idx}") 

In [ ]:
df['Median Household Income (In 2021 Inflation Adjusted Dollars)']

In [ ]:
# county_boundary_death_senior_gdf
# senior_rate
# death_rate
# covid_dataframe

covid_dataframe['death_rate'] = covid_dataframe['deaths'] / covid_dataframe['cases']
death_rate = covid_dataframe.groupby('fips')['death_rate'].last()

death_rate